# I. Fitting Data


In [ ]:
library(neuralnet)


In [ ]:
set.seed(123)


In [ ]:
# simulacion de datos
n <- 300
x <- seq(0, 2*pi, length.out = n)
epsilon <- rnorm(n, 0, 0.1)
y <- sin(x) + epsilon

datos <- data.frame(x = x, y = y)

png("simulacion_R.png", width = 800, height = 400)
plot(x, y, pch = 19, col = "gray",
     main = "Simulación: y = sin(x) + error",
     xlab = "x", ylab = "y")
dev.off()


In [ ]:
# entrenamiento
activaciones <- list(
  logistic = "logistic",
  tanh = "tanh"
)

modelos <- list()
predicciones <- list()
mses <- c()

for (act in names(activaciones)) {
  cat("\nEntrenando red con activacion:", act, "\n")

  modelo <- neuralnet(
    y ~ x,
    data = datos,
    hidden = c(20, 20, 20),  # más rápido que 50,50,50
    act.fct = activaciones[[act]],
    linear.output = TRUE,
    stepmax = 1e6
  )

  modelos[[act]] <- modelo

  pred <- compute(modelo, data.frame(x = x))$net.result
  predicciones[[act]] <- pred
  mses[act] <- mean((pred - y)^2)

  png(paste0("ajuste_", act, "_R.png"), width = 800, height = 400)
  plot(x, y, pch = 19, col = "gray",
       main = paste("Ajuste NN activacion:", act),
       xlab = "x", ylab = "y")
  lines(x, pred, col = "red", lwd = 2)
  dev.off()
}

cat("\nMSE obtenidos:\n")
print(mses)


En esta simulación, la activación **tanh** obtuvo el menor MSE, por lo que corresponde a la red que mejor se ajusta a los datos. Esto refleja que *tanh* modela mejor el patrón suave de la función seno, mientras que *logistic* resulta algo más rígida en los extremos del rango.


En este entorno de R no fue posible entrenar redes con activación **ReLU** ni construir una red mixta con funciones distintas por capa. Las librerías disponibles (como `neuralnet`, `nnet` y `RSNNS`) solo aceptan activaciones como *logistic* o *tanh*, y los intentos de usar ReLU generaron errores de convergencia o mensajes del kernel indicando que la función no es válida. Por ello, en R solo se reportan los resultados para *logistic* y *tanh*, mientras que las arquitecturas con ReLU y mezcla de activaciones se implementan en los otros lenguajes.


## II. Learning-rate

El *learning rate* es un parámetro que controla el tamaño de los pasos con los que una red neuronal ajusta sus pesos durante el entrenamiento. Un valor muy pequeño hace que el aprendizaje sea lento, mientras que uno demasiado grande puede provocar inestabilidad y que la red no converja adecuadamente. Su elección influye directamente en la calidad y la velocidad del ajuste del modelo.


In [ ]:
# simulación de datos
n <- 300
x <- seq(0, 2*pi, length.out = n)
epsilon <- rnorm(n, mean = 0, sd = 0.1)
y <- sin(x) + epsilon
datos <- data.frame(x = x, y = y)


In [ ]:
library(neuralnet)


In [ ]:
lrs <- c(0.0001, 0.001, 0.01, 0.1)
preds <- list()


In [ ]:
for (lr in lrs) {
  cat("\nEntrenando NN con learning rate:", lr, "\n")
  modelo <- neuralnet(
    y ~ x,
    data = datos,
    hidden = 50,
    act.fct = "tanh",
    linear.output = TRUE,
    learningrate = lr,
    stepmax = 1e6
  )
  pred <- compute(modelo, data.frame(x = x))$net.result
  preds[[as.character(lr)]] <- pred
}


In [ ]:
png("learning_rate_R.png", width = 800, height = 400)
plot(x, y, pch = 19, col = "gray",
     main = "Comparación de learning rates (1 hidden layer)",
     xlab = "x", ylab = "y")
cols <- c("red", "blue", "green", "purple")
i <- 1
for (lr in lrs) {
  lines(x, preds[[as.character(lr)]], col = cols[i], lwd = 2)
  i <- i + 1
}
legend("bottomleft", legend = paste("lr =", lrs),
       col = cols, lwd = 2, bty = "n")
dev.off()


In [ ]:
capas <- list("2 capas" = c(50, 50),
              "3 capas" = c(50, 50, 50))


In [ ]:
for (nombre in names(capas)) {
  cat("\n\n##### Entrenando:", nombre, "#####\n")
  preds_capas <- list()

  # Reducir tamaño para evitar que neuralnet colapse
  hidden_config <- rep(20, length(capas[[nombre]]))

  for (lr in lrs) {
    cat("Learning rate:", lr, "\n")

    modelo <- neuralnet(
      y ~ x,
      data = datos,
      hidden = hidden_config,
      act.fct = "tanh",
      linear.output = TRUE,
      learningrate = lr,
      stepmax = 3e5
    )

    pred <- compute(modelo, data.frame(x = x))$net.result
    preds_capas[[as.character(lr)]] <- pred
  }

  png(paste0("learning_rate_", gsub(" ", "_", nombre), ".png"),
      width = 800, height = 400)

  plot(x, y, pch = 19, col = "gray",
       main = paste("Learning rate -", nombre),
       xlab = "x", ylab = "y")

  cols <- c("red", "blue", "green", "purple")
  i <- 1
  for (lr in lrs) {
    lines(x, preds_capas[[as.character(lr)]],
          col = cols[i], lwd = 2)
    i <- i + 1
  }

  legend("bottomleft", legend = paste("lr =", lrs),
         col = cols, lwd = 2, bty = "n")

  dev.off()
}


Se observa que el desempeño del modelo depende tanto del *learning rate* como de la profundidad de la red. Con un solo nivel oculto, los learning rates intermedios (0.001 y 0.01) producen los ajustes más estables, mientras que tasas muy bajas generan aprendizaje lento y tasas altas provocan oscilaciones. Al aumentar el número de capas a dos y tres, la red se vuelve más sensible: los learning rates altos dejan de funcionar y solo los valores más pequeños (0.0001 o 0.001) logran converger sin inestabilidad. En conjunto, los resultados muestran que redes más profundas requieren learning rates más pequeños para garantizar un entrenamiento estable.
